# *Laplacian* analysis

In [1]:
# SPECIFY WORKING DIRECTORY
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import numpy as np
import pandas as pd

## IMORT FUNCTIONS FROM OTHER FILES
from spectral_clustering import *
from parameter_fitting import *
from stability_analysis import *
from smoothness_analysis import *

So, I introduced the graph Laplacian as an object that meassures smoothness and variance throughout the graph. The professor then asked me for a concrete example of when the Laplacian acts like the variance and I could not give him an answer... Let's try and find a concrete example.

### **Theoretical Framework**:

For a function on the nodes $f: \mathcal{V} \rightarrow \mathbb{R}$ (this might be thought of as $f_i:=f(i)$), the **Laplacian quadratic form** (as stated in the paper) is
$$\forall f \in \mathbb{R}^n : f^\top L f = \frac{1}{2} \sum_{i,j}w_{i,j}(f_i-f_j)^2.$$

This shows two main things:

- If neighbors $i,j$ have similar values ($f_i \approx f_j$), then $(f_i-f_j)^2$ is really small $\rightarrow$ low 'energy'.
- If rhe signal oscillates a lot between neighbors, the energy explodes.

A signal (graph signal) is a function on the nodes as described above ($f$). The Laplacian acts on $f$ as
$$
(Lf)_i = d_i f_i - \sum_j w_{ij} f_j = \sum_j w_{ij}\,(f_i - f_j),
$$

which measures how different the value at node $i$ is from its neighbors. If neighboring nodes have similar values, $Lf \approx 0$.

It is really important tonote the *laplacian quadratic form* (above), which is a weighted sum of squared differences across edges, so:

- If $f$ is **smooth** on the graph (neighbors have similar values), then $f^\top L f$ is small.
- If $f$ **oscillates** strongly between neighbors, then $f^\top L f$ is large.

So, $f^\top L f$ plays the role of a **graph-structured variance** or **roughness measure**.

### **Experimentation**:

Recaping what was stated in the **Theoretical Framework**, 
- A signal on nodes = a vector of values $f_i$
- $Lf$ = how “unsmooth” the signal is locally.
- $f^\top Lf$ = total “unsmoothness energy”: heavilyrelated to variance, but structured by the graph.

So, to make the ‘Laplacian as variance/smoothness’ interpretation concrete, I ran two small experiments...

- A: **Variance vs 'Laplacian energy'** $E(f):=f^\top Lf$: 
    - Choose some graphs: complete graph $K_n$, Path graph $P_n$ (lind of len. $n$), maybe annother?
    - For each graph, we do:
        - sample a random vector $f \sim \mathcal{N}(0,I) \in \mathbb{R}^n$
        - center it: $\hat{f} = f - \mathbf{1}\bar{f}$, where $\bar{f} = \frac{1}{n} \sum_i f_i$
        - compute the usual variance: $\operatorname{Var}(\hat{f}) = \frac{1}{n} \sum_i \hat{f}^2$ and Laplacian energy $E(\hat{f})$
        - inspect/plto

- B: **Smooth vs Non-smooth signals** 
    - Still thinking on this

#### Experiment A:

In [10]:
complete_graph = W_complete_graph(5)
D, L, L_rw = build_laplacians_from_W(complete_graph)

In [11]:
df, _ = experiment_variance_vs_energy(n=10, n_trials=10)

               var     energy       ratio
graph                                    
complete  0.721681  72.168073  100.000000
path      1.025827  16.795526   18.173404


In [23]:
df

,graph,trial,var,energy,ratio
0,complete,0,0.412355,41.235544,100.000000
1,complete,1,0.843366,84.336583,100.000000
2,complete,2,0.810411,81.041073,100.000000
3,complete,3,0.694770,69.476985,100.000000
4,complete,4,0.687179,68.717911,100.000000
5,complete,5,0.769601,76.960145,100.000000
6,complete,6,0.829427,82.942725,100.000000
7,complete,7,0.392936,39.293568,100.000000
8,complete,8,0.761520,76.152009,100.000000
9,complete,9,1.015242,101.524183,100.000000


In [29]:
tipo, varianza, energia = df['graph'][10], df['var'][10], df['energy'][10]

In [34]:
ratio_test = energia / varianza
ratio_test

np.float64(8.497802587894382)

Note that as we suspected, the laplacian enery is the same as a scaled version of the varaince!